In [9]:
%load_ext autoreload
%autoreload 2
import sys
import gc

from multiprocessing import cpu_count, Pool
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data_utils

from tqdm import tqdm

sys.path.append('../')
from wiki.utils import clock
from wiki import rnn, rnn_predict, newphet, val
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
torch.manual_seed(1)
torch.cuda.set_device(0)

In [11]:
batch_size = 1024
pred_len = 62

In [12]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_2.csv').fillna(0)

In [13]:
X = train_df.drop('Page', axis=1).values
X, scaler = rnn.scale_values(X)

69 here because pytorch was throwing hissy fit because (conjecture via pytorch forums) the sequence length was >600

In [14]:
trainloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,69:-2*pred_len,:]).float(),
        torch.from_numpy(X[:,-2*pred_len:-pred_len,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

In [15]:
valloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:-pred_len,:]).float(),
        torch.from_numpy(X[:,-pred_len:,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

In [16]:
loss_func = nn.L1Loss()
model = rnn.RNN(loss_func=loss_func).cuda()

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
save_best_path = base_dir+'rnn_stage2_v2_lr1_more.mdl'
with clock():
    model.fit(trainloader, valloader, optimizer=optimizer, num_epochs=20, save_best_path=save_best_path)


EPOCH 1
Running average loss: 0.308078
VALIDATION LOSS: 0.345460
Elapsed time 155.6518201828003 seconds

EPOCH 2
Running average loss: 0.294705
VALIDATION LOSS: 0.355449
Elapsed time 157.21379351615906 seconds

EPOCH 3
Running average loss: 0.279476
VALIDATION LOSS: 0.317788
Elapsed time 157.7180187702179 seconds

EPOCH 4
Running average loss: 0.276255
VALIDATION LOSS: 0.318387
Elapsed time 159.20266699790955 seconds

EPOCH 5
Running average loss: 0.270387
VALIDATION LOSS: 0.349600
Elapsed time 160.13787603378296 seconds

EPOCH 6
Running average loss: 0.260348
VALIDATION LOSS: 0.314421
Elapsed time 160.25575351715088 seconds

EPOCH 7
Running average loss: 0.261328
VALIDATION LOSS: 0.325439
Elapsed time 160.21467971801758 seconds

EPOCH 8
Running average loss: 0.266648
VALIDATION LOSS: 0.306064
Elapsed time 160.27470207214355 seconds

EPOCH 9
Running average loss: 0.258198
VALIDATION LOSS: 0.304726
Elapsed time 160.36223769187927 seconds

EPOCH 10
Running average loss: 0.258665
VALIDAT

In [18]:
save_best_path = base_dir+'rnn_stage2_v2_lr1_more.mdl'
model.load_state_dict(torch.load(save_best_path))

FileNotFoundError: [Errno 2] No such file or directory: '../data/rnn_stage2_v2_lr1_more.mdl'

In [19]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
save_best_path = base_dir+'rnn_stage2_v2_lr2_more.mdl'
with clock():
    model.fit(trainloader, valloader, optimizer=optimizer, num_epochs=10, save_best_path=save_best_path)


EPOCH 1
Running average loss: 0.239370
VALIDATION LOSS: 0.279059
Elapsed time 159.48737835884094 seconds

EPOCH 2
Elapsed time 77.59359741210938 seconds
Elapsed time 237.08160591125488 seconds


KeyboardInterrupt: 

In [20]:
save_best_path = base_dir+'rnn_stage2_v2_lr2_more.mdl'
model.load_state_dict(torch.load(save_best_path))

In [ ]:
outputs, targets, sequences = model.predict(valloader)

In [ ]:
_, predictions = rnn_predict.combine_prediction_data(outputs, targets, sequences)

In [ ]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_2.csv')
X = train_df.drop('Page', axis=1).values

In [ ]:
predictions = scaler.inverse_transform(predictions.T).T
true = X

In [ ]:
smapes = val.smape(true[:,-60:], predictions[:,-60:], axis=1)
smapes_clipped = val.smape(true[:,-60:], predictions[:,-60:].round().clip(0), axis=1)

In [ ]:
np.nanmean(smapes), np.nanmean(smapes_clipped)